<a href="https://colab.research.google.com/github/truongthuanr/transfer/blob/main/NLP_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CommonLit Readability problem

### Check for GPU

In [ ]:
!nvidia-smi -L

### Line wrap setting for notebook

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Get helper function

In [ ]:
# Get the heper function 
!wget https://raw.githubusercontent.com/truongthuanr/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get text data set
The data set we going to using is from Kagle "CommonLit Readability Prize" Competetion
https://www.kaggle.com/c/commonlitreadabilityprize/data


In [ ]:
# Unzip data
# !unzip "/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/commonlitreadabilityprize.zip" -d "/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/"

## Visualizing a text dataset

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/test.csv")

In [ ]:
train_df.columns

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [ ]:
# train_df_shuffled.head()

In [ ]:
# How the target look like
sns.histplot(train_df.target, bins=20,color='navy')

In [ ]:
# How many total sample?
len(train_df), len(test_df)

In [ ]:
# visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["excerpt","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}")
  print(f"{text}\n")
  print("---")

## Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Use train_test_split to split trainind data into training and validation sets
train_excerpt, val_excerpt, train_taget, val_target = train_test_split(train_df_shuffled["excerpt"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=41)

In [ ]:
len(train_excerpt), len(train_taget), len(val_excerpt), len(val_target)

## Converting text in to number
When dealing with the text problem, one of the first things we have to do before you can build a model is to convert our text to number.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector canbe defined and hthis embedding can be learned)

### Text vectorization (Tokenization)

Info: https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization

In [ ]:
# train_excerpt[:4]

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many word in the vocabulary
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int", # how to map tokens to number
                                    output_sequence_length=None, # how long do we want our sequences to be
                                    pad_to_max_tokens=True)

In [ ]:
# Find the avarage number of tokens (words) in the training excerpt
round(sum([len(i.split()) for i in train_excerpt])/len(train_excerpt))

In [ ]:
# Setup the vectorization variables 
max_vocab_length = 10000 # max number of words to have in our vocabolary
max_length = 173 # max length our excerpt will be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text Vectorizer to the text training data